In [1]:
#Ning
#03 November 2019
import json

In [2]:
#load the json file into a dictionary
jsonfp = open("toygrammar.json", "r")
json_grammar = json.load(jsonfp)

In [3]:
#check that we listed the information correctly
json_grammar

{'I': 'S',
 'NTR': [['S', 'NP', 'VP', 1.0],
  ['NP', 'DET', 'N', 0.4],
  ['NP', 'DET', 'NPP', 0.6],
  ['NPP', 'N', 'PP', 1.0],
  ['VP', 'V', 'NP', 0.3],
  ['VP', 'V', 'NPPP', 0.7],
  ['NPPP', 'NP', 'PP', 1.0],
  ['PP', 'P', 'NP', 1.0]],
 'TR': [['DET', 'the', 1.0],
  ['N', 'girl', 0.3],
  ['N', 'man', 0.5],
  ['N', 'telescope', 0.2],
  ['V', 'sees', 0.6],
  ['V', 'watches', 0.4],
  ['P', 'with', 1.0]]}

In [4]:
#reordering the dictionary in order to comply with section 1 subpoint 6: 
"""It might be useful to store the rules as dictionaries of dictionaries, using the left
hand side as the first key, the right hand side as the second key, and the probability
as the value."""
grammar = {}
unit_reverse = {}
word_p = {}

for item in json_grammar["NTR"]: #goes through all non-terminating rules
    
    #define the parts
    lhs = "{0}".format(item[0])
    
    if not lhs in grammar: #if we don't already have this lhs hashed
        rhs = {} #create new
        rhs["{0} {1}".format(item[1], item[2])] = item[3] #define the rhs
        grammar[lhs] = rhs #hash it into grammar
    else:
        grammar[lhs]["{0} {1}".format(item[1], item[2])] = item[3] #just hash

for item in json_grammar["TR"]: #just copy over the terminating rules?
    
    #define the parts
    lhs = "{0}".format(item[0])
    
    if not lhs in grammar:
        rhs = {} #set new
        rhs["{0}".format(item[1])] = item[2] #define the rhs
        grammar[lhs] = rhs #hashes into the grammar
    else:
        grammar[lhs]["{0}".format(item[1])] = item[2] #hashes into the grammar

#Note that we also want the terminating words as keys so that searching backwards is doable
for item in json_grammar["TR"]:
    word = "{0}".format(item[1])
    reverse = "{0}".format(item[0])
    unit_reverse[word] = reverse

#for probs
for item in json_grammar["TR"]:
    word = "{0}".format(item[1])
    prob = item[2]
    word_p[word] = prob

In [5]:
#confirms that the grammar is stored as desired
print(grammar)

{'S': {'NP VP': 1.0}, 'NP': {'DET N': 0.4, 'DET NPP': 0.6}, 'NPP': {'N PP': 1.0}, 'VP': {'V NP': 0.3, 'V NPPP': 0.7}, 'NPPP': {'NP PP': 1.0}, 'PP': {'P NP': 1.0}, 'DET': {'the': 1.0}, 'N': {'girl': 0.3, 'man': 0.5, 'telescope': 0.2}, 'V': {'sees': 0.6, 'watches': 0.4}, 'P': {'with': 1.0}}


In [12]:
#following the pseudo code implementation
def cyk_me_up_fam(sentence, grammar, unit_reverse, word_p):
    
    '''
    sentence should be an array input of words in order
    grammar should be a dictionary set up properly
    '''
    
    n = len(sentence) #input size
    grid = [x[:] for x in [[None]*n]*n]
    probs = []
    
    #prelims -> sets the first row of unit prods
    for q in range(0,n):
        grid[0][q] = unit_reverse[sentence[q]] #first row contains the marker for unit prod
    
    #elims -> grey-box checks if lhs can produce f_val s_val 

    #potential productions, denoted f_val, s_val
    for l in range(2, n+1):
        for s in range(1, n-l+2):
            for p in range(1, l-1+1):

                f_val = grid[p-1][s-1] #first run should be p,s = 0,0 and l-p,s+p = 0,1
                s_val = grid[l-p-1][s+p-1] #second run p,s = 0,1 and l-p,s+p = 0,2

                potpro = "{0} {1}".format(f_val, s_val) #potential production

                #check lhs' dictionaries to see if there exist a key that corresponds
                lhs = list(grammar.keys())
                for keys in lhs:
                    rhs = list(grammar[keys].keys())
                    if potpro in rhs: #match, align the box
                        grid[l-1][s-1] = keys
                        
                        #do book-keepng here for probabilities
                        probs.append(grammar[keys][potpro])
                        
    #if no grammar
    if not 'S' in grid[n-1]:
        return "Sentence not in grammar"
    
    #return the tree and probability here
    ret = []
    for col in range(0, n): #for each slot down the grid
        for row in range(n-1, -1, -1):
            if grid[row][col] != None:
                ret.append("{0}".format(grid[row][col])) 
    
    #account for the final probability of words
    for word in sentence:
        probs.append(word_p[word])
    
    big_p = 1
    for item in probs:
        big_p *= item
    
    #we can figure out how to properly organize tree in a bit
    return (ret, big_p)

In [13]:
cyk_me_up_fam(["the", "girl", "sees", "the", "telescope"], grammar, unit_reverse, word_p)

(['S', 'NP', 'DET', 'N', 'VP', 'V', 'NP', 'DET', 'N'], 0.0017280000000000002)

In [14]:
cyk_me_up_fam(["the", "man", "the", "girl"], grammar, unit_reverse, word_p)

'Sentence not in grammar'

In [15]:
cyk_me_up_fam(["the", "man", "sees", "the", "man", "with", "the", "telescope"], grammar, unit_reverse, word_p)

(['S',
  'S',
  'NP',
  'DET',
  'N',
  'VP',
  'VP',
  'V',
  'NPPP',
  'NP',
  'DET',
  'NPP',
  'N',
  'PP',
  'P',
  'NP',
  'DET',
  'N'],
 0.00024192000000000004)